Teste para achar jogador que cruzou

In [1836]:
import pandas as pd
import scipy.stats as st
import numpy as np
import matplotlib.pyplot as plt
import math

In [1837]:
# Leitura do arquivo csv
df = pd.read_csv('dados/dados_brutos.csv')

In [1838]:
# Organizar dados por frame e time do jogador
df = df.sort_values(by=['FRAME', 'TIME_JOGADOR'])

In [1839]:
# Colunas do dataframe
df.columns

Index(['TEMPO_JOGO', 'FRAME', 'ID_JOGADOR', 'NOME_JOGADOR', 'POSICAO_JOGADOR',
       'TIME_JOGADOR', 'OBJETO_OBSERVADO', 'X', 'Y', 'Z', 'VELOCIDADE',
       'TIME_POSSE'],
      dtype='object')

In [1840]:
df_bola = df[df['OBJETO_OBSERVADO'] == 'bola']

df_bola["DENTRO_AREA_DIREITA"] = 0
df_bola["DENTRO_AREA_ESQUERDA"] = 0

area_direita = (df_bola["X"] > 101.5) & (df_bola["Y"] > 17.85) & (df_bola["Y"] < 58.15)
df_bola.loc[area_direita, "DENTRO_AREA_DIREITA"] = 1

area_esquerda = (df_bola["X"] < 16.5) & (df_bola["Y"] > 17.85) & (df_bola["Y"] < 58.15)
df_bola.loc[area_esquerda, "DENTRO_AREA_ESQUERDA"] = 1

C:\Users\Daniel\AppData\Local\Temp\ipykernel_9348\4292862974.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bola["DENTRO_AREA_DIREITA"] = 0
C:\Users\Daniel\AppData\Local\Temp\ipykernel_9348\4292862974.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bola["DENTRO_AREA_ESQUERDA"] = 0


In [1841]:
df_bola = df_bola.sort_values(by=['TEMPO_JOGO', 'FRAME'])
df_bola.reset_index(drop=True, inplace=True)
df_bola

,TEMPO_JOGO,FRAME,ID_JOGADOR,NOME_JOGADOR,POSICAO_JOGADOR,TIME_JOGADOR,OBJETO_OBSERVADO,X,Y,Z,VELOCIDADE,TIME_POSSE,DENTRO_AREA_DIREITA,DENTRO_AREA_ESQUERDA
0,1,0,NaN,NaN,NaN,NaN,bola,59.12,38.29,0.0,41.19,PAL,0,0
1,1,1,NaN,NaN,NaN,NaN,bola,59.31,39.28,0.0,41.68,PAL,0,0
2,1,2,NaN,NaN,NaN,NaN,bola,57.81,39.13,0.0,40.98,PAL,0,0
3,1,3,NaN,NaN,NaN,NaN,bola,57.02,39.45,0.0,40.77,PAL,0,0
4,1,4,NaN,NaN,NaN,NaN,bola,57.91,40.94,0.0,41.77,PAL,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146677,2,76213,NaN,NaN,NaN,NaN,bola,67.96,36.98,0.0,0.00,PAL,0,0
146678,2,76214,NaN,NaN,NaN,NaN,bola,66.96,36.44,0.0,0.00,PAL,0,0
146679,2,76215,NaN,NaN,NaN,NaN,bola,66.95,36.43,0.0,0.00,PAL,0,0
146680,2,76216,NaN,NaN,NaN,NaN,bola,66.53,36.20,0.0,0.00,PAL,0,0


In [1842]:
# Palmeiras atacando a área direita

df_pal_ataque_direita = df_bola[
    (df_bola['TIME_POSSE'] == 'PAL') & 
    (df_bola['VELOCIDADE'] >= 1.5) & 
    (df_bola['TEMPO_JOGO'] == 1) & 
    (df_bola['X'] < 118) & 
    (df_bola['X'] >= 86.5) & 
    (df_bola['Y'] < 76)
]

df_pal_ataque_direita.reset_index(drop=True, inplace=True)

areas_direita = {
    'E1.2':{
        'X0': 86.5,
        'X1': 101.5,
        'Y0': 0,
        'Y1': 17.85,
        'num': 0
    },
    'E1.1':{
        'X0': 86.5,
        'X1': 101.5,
        'Y0': 17.85,
        'Y1': 38,
        'num': 0
    },
    'E2.1':{
        'X0': 101.5,
        'X1': 109.75,
        'Y0': 0,
        'Y1': 17.85,
        'num': 0
    },
    'E2.2':{
        'X0': 109.75,
        'X1': 118,
        'Y0': 0,
        'Y1': 17.85,
        'num': 0
    },
    'E3':{
        'X0': 101.5,
        'X1': 109.75,
        'Y0': 17.85,
        'Y1': 27.92,
        'num': 0
    },
    'D1.1':{
        'X0': 86.5,
        'X1': 101.5,
        'Y0': 38,
        'Y1': 58.15,
        'num': 0
    },
    'D1.2':{
        'X0': 86.5,
        'X1': 101.5,
        'Y0': 58.15,
        'Y1': 76,
        'num': 0
    },
    'D2.1':{
        'X0': 101.5,
        'X1': 109.75,
        'Y0': 58.15,
        'Y1': 76,
        'num': 0
    },
    'D2.2':{
        'X0': 109.75,
        'X1': 118,
        'Y0': 58.15,
        'Y1': 76,
        'num': 0
    },
    'D3':{
        'X0': 101.5,
        'X1': 109.75,
        'Y0': 48.07,
        'Y1': 58.15,
        'num': 0
    }
}

In [1843]:
def analisa_frames(frames):
    frames_analisados = []
    for i in range(len(frames)):
        if i+1 == len(frames):
            break
        if frames[i+1] - frames[i] < 10:
            frames_analisados.append(frames[i])
    return frames_analisados

In [1844]:
cruzamentos = 0

frames = []

for index, row in df_pal_ataque_direita.iterrows():
    if index+1 == len(df_pal_ataque_direita):
        break
    if df_pal_ataque_direita.iloc[index]['DENTRO_AREA_DIREITA'] == 0 and df_pal_ataque_direita.iloc[index+1]['DENTRO_AREA_DIREITA'] == 1:
        sequencia_de_1 = 0
        for i in range(1, 21):
            if index+i == len(df_pal_ataque_direita): # talvez colocar um -1 aqui para ele ver o último frame ???
                break
            if df_pal_ataque_direita.iloc[index+i]['DENTRO_AREA_DIREITA'] == 1:
                sequencia_de_1 += 1
            if sequencia_de_1 >= 20: # bola deve estar na área por pelo menos 0.8 segundo
                for area in areas_direita:
                    if df_pal_ataque_direita.iloc[index-50]['X'] > areas_direita[area]['X0'] and df_pal_ataque_direita.iloc[index-50]['X'] < areas_direita[area]['X1'] and df_pal_ataque_direita.iloc[index-50]['Y'] > areas_direita[area]['Y0'] and df_pal_ataque_direita.iloc[index-50]['Y'] < areas_direita[area]['Y1']:
                        areas_direita[area]['num'] += 1
                        cruzamentos += 1
                        frames.append(df_pal_ataque_direita.iloc[index+1]['FRAME']) # frame quando a bola entra na área
                        break
                break

print(f'Cruzamentos: {cruzamentos}')
print(f'Frames: {frames}')

# Para comparar com 'frames'
frames_analisados = analisa_frames(frames)
print(f'Frames Analisados: {frames_analisados}')

Cruzamentos: 19
Frames: [8189, 14138, 15308, 19367, 19484, 20363, 22567, 27621, 29742, 39832, 40645, 48331, 49319, 49455, 50587, 52091, 52427, 62235, 67044]
Frames Analisados: []


In [1845]:
df_palmeiras = df[(df['TIME_JOGADOR'] != 'RED')]
df_bragantino = df[(df['TIME_JOGADOR'] != 'PAL')]
df_palmeiras_t1 = df_palmeiras[df_palmeiras['TEMPO_JOGO'] == 1]
jogadores_cruzamentos = {} # Dicionário com os jogadores que cruzaram a bola / frames que cruzaram a bola
velocidades_maximas = {} # Dicionário com as velocidades máximas da bola em cada frame que ocorre um cruzamento


frames_novos = [8183,14134,15268,19341,19484,20358,22566,27600,29742,39832,40628,48329,49309,49415,50545,52088,52423,62186,67040]

for frame in frames_novos:
    x_bola = 0
    y_bola = 0
    velocidade_bola = 0
    distancia_entre_jogador_e_bola_antes = 100 # Valor inicial alto para que qualquer distância seja menor que ele
    distancia_entre_jogador_e_bola_depois = 100 # Valor inicial alto para que qualquer distância seja menor que ele
    
    # DataFrame com os dados da bola no intervalo do cruzamento (frame e 2 segundos antes)
    df_frame_bola = df_palmeiras_t1[(df_palmeiras_t1['FRAME'] <= frame) & (df_palmeiras_t1['FRAME'] >= frame-50) & (df_palmeiras_t1['OBJETO_OBSERVADO'] == 'bola')]

    for index, row in df_frame_bola.iterrows():
        if row['VELOCIDADE'] > velocidade_bola:
            velocidade_bola = row['VELOCIDADE']
            frame_que_cruzou = row['FRAME']
            # Coordenadas da bola no momento do cruzamento
            x_bola = row['X']
            y_bola = row['Y']

    # Velocidades máximas atingidas pela bola em cada cruzamento (momento em que o jogador toca na bola)
    velocidades_maximas[frame_que_cruzou] = velocidade_bola

    # Mesmo DataFrame, mas com os dados dos jogadores
    df_frame_jogadores = df_palmeiras_t1[(df_palmeiras_t1['FRAME'] <= frame) & (df_palmeiras_t1['FRAME'] >= frame-50) & (df_palmeiras_t1['OBJETO_OBSERVADO'] == 'jogador')]

    for index, row in df_frame_jogadores.iterrows():
        jog = [row['X'], row['Y']]
        bola = [x_bola, y_bola]
        distancia_medida = math.dist(jog, bola) # Distância entre o jogador e a bola: distância de dois pontos
        if distancia_medida < distancia_entre_jogador_e_bola_antes:
            distancia_entre_jogador_e_bola_antes = distancia_medida
            jogador = row['NOME_JOGADOR'] # Nome do jogador que cruzou
            jog_antes = jog
        
    df_frame_bola_depois = df_palmeiras_t1[(df_palmeiras_t1['FRAME'] == frame+10) & (df_palmeiras_t1['OBJETO_OBSERVADO'] == 'bola')]
    df_frame_jogadores_depois = df_palmeiras_t1[(df_palmeiras_t1['FRAME'] == frame+10) & (df_palmeiras_t1['OBJETO_OBSERVADO'] == 'jogador')]
    df_jogador = df_frame_jogadores_depois[df_frame_jogadores_depois['NOME_JOGADOR']== jogador]
    x = df_jogador['X'].iloc[0]
    y = df_jogador['Y'].iloc[0]
    posicao_jogador_depois = [x,y]
    x_bola_depois = df_frame_bola_depois['X'].iloc[0]
    y_bola_depois = df_frame_bola_depois['Y'].iloc[0]
    posicao_bola_depois = [x_bola_depois,y_bola_depois]
    distancia_medida_depois = math.dist(posicao_jogador_depois, posicao_bola_depois)
    if distancia_medida_depois >= (distancia_entre_jogador_e_bola_antes + 3):
        print('1')
    else:
        print('2')
    print(distancia_entre_jogador_e_bola_antes, distancia_medida_depois)



,TEMPO_JOGO,FRAME,ID_JOGADOR,NOME_JOGADOR,POSICAO_JOGADOR,TIME_JOGADOR,OBJETO_OBSERVADO,X,Y,Z,VELOCIDADE,TIME_POSSE
1542162,1,67050,100016.0,Vanderlan,Lateral Esquerdo,PAL,jogador,115.93,0.48,NaN,7.94,NaN


In [1846]:
df_palmeiras = df[(df['TIME_JOGADOR'] != 'RED')]
df_bragantino = df[(df['TIME_JOGADOR'] != 'PAL')]

In [1847]:
df_palmeiras_t1 = df_palmeiras[df_palmeiras['TEMPO_JOGO'] == 1]

In [1848]:
# Dicionários
jogadores_cruzamentos = {} # Dicionário com os jogadores que cruzaram a bola / frames que cruzaram a bola
velocidades_maximas = {} # Dicionário com as velocidades máximas da bola em cada frame que ocorre um cruzamento

In [1849]:
# Análise de quem realizou cada cruzamento contado em 'frames'
for frame in frames:
    x_bola = 0
    y_bola = 0
    velocidade_bola = 0
    distancia_entre_jogador_e_bola = 100 # Valor inicial alto para que qualquer distância seja menor que ele
    
    # DataFrame com os dados da bola no intervalo do cruzamento (frame e 2 segundos antes)
    df_frame_bola = df_palmeiras_t1[(df_palmeiras_t1['FRAME'] <= frame) & (df_palmeiras_t1['FRAME'] >= frame-50) & (df_palmeiras_t1['OBJETO_OBSERVADO'] == 'bola')]

    for index, row in df_frame_bola.iterrows():
        if row['VELOCIDADE'] > velocidade_bola:
            velocidade_bola = row['VELOCIDADE']
            frame_que_cruzou = row['FRAME']
            # Coordenadas da bola no momento do cruzamento
            x_bola = row['X']
            y_bola = row['Y']

    # Velocidades máximas atingidas pela bola em cada cruzamento (momento em que o jogador toca na bola)
    velocidades_maximas[frame_que_cruzou] = velocidade_bola

    # Mesmo DataFrame, mas com os dados dos jogadores
    df_frame_jogadores = df_palmeiras_t1[(df_palmeiras_t1['FRAME'] <= frame) & (df_palmeiras_t1['FRAME'] >= frame-50) & (df_palmeiras_t1['OBJETO_OBSERVADO'] == 'jogador')]

    for index, row in df_frame_jogadores.iterrows():
        jog = [row['X'], row['Y']]
        bola = [x_bola, y_bola]
        distancia_medida = math.dist(jog, bola) # Distância entre o jogador e a bola: distância de dois pontos
        if distancia_medida < distancia_entre_jogador_e_bola:
            distancia_entre_jogador_e_bola = distancia_medida
            jogador = row['NOME_JOGADOR'] # Nome do jogador que cruzou

    if jogador in jogadores_cruzamentos:
        jogadores_cruzamentos[jogador].append(frame_que_cruzou)
    else:
        jogadores_cruzamentos[jogador] = [frame_que_cruzou]

print(velocidades_maximas)
print(jogadores_cruzamentos)




{8183: 39.32, 14134: 39.65, 15268: 31.78, 19341: 36.36, 19484: 28.95, 20358: 41.94, 22566: 41.65, 27600: 41.92, 29742: 36.12, 39832: 35.1, 40628: 41.9, 48329: 41.93, 49309: 41.75, 49415: 41.8, 50545: 33.4, 52088: 41.98, 52423: 39.99, 62186: 41.95, 67040: 41.97}
{'Raphael Veiga': [8183, 19484, 20358, 22566, 40628, 49309], 'Artur': [14134, 19341, 27600, 52423], 'Rony': [15268, 50545], 'Vanderlan': [29742, 62186, 67040], 'Dudu': [39832, 48329, 52088], 'Gustavo Gomez': [49415]}


In [1850]:

def frames_para_tempo(frames):
    tempos = []
    for frame in frames:
        minutos = frame // (25 * 60)
        segundos = (frame // 25) % 60
        tempos.append((minutos, segundos))
    return tempos

frames = [8189, 14138, 15308, 19367, 19484, 20363, 22567, 27621, 29742, 39832, 40645, 48331, 49319, 49455, 50587, 52091, 52427, 62235, 67044]

frames_tempo = []
tempos = frames_para_tempo(frames)

for i, tempo in enumerate(tempos, start=1):
    frames_tempo.append(f"Cruzamento {i}: {tempo[0]} minutos e {tempo[1]} segundos")

print(frames_tempo)

cruzamentos = [
'Cruzamento 1: 5 minutos e 27 segundos', #OK
'Cruzamento 2: 9 minutos e 25 segundos', #ERRADO
'Cruzamento 3: 10 minutos e 12 segundos',#OK
'Cruzamento 4: 12 minutos e 54 segundos',#OK
'Cruzamento 5: 12 minutos e 59 segundos',#OK    
'Cruzamento 6: 13 minutos e 34 segundos',#OK
'Cruzamento 7: 15 minutos e 2 segundos', #OK
'Cruzamento 8: 18 minutos e 24 segundos',#OK
'Cruzamento 9: 19 minutos e 49 segundos',#OK
'Cruzamento 10: 26 minutos e 33 segundos',#OK
'Cruzamento 11: 27 minutos e 5 segundos',#ESCANTEIO
'Cruzamento 12: 32 minutos e 13 segundos',#OK
'Cruzamento 13: 32 minutos e 52 segundos',#ESCANTEIO
'Cruzamento 14: 32 minutos e 58 segundos',#OK
'Cruzamento 15: 33 minutos e 43 segundos',#ERRADO
'Cruzamento 16: 34 minutos e 43 segundos',#OK
'Cruzamento 17: 34 minutos e 57 segundos',#OK
'Cruzamento 18: 41 minutos e 29 segundos',#OK
'Cruzamento 19: 44 minutos e 41 segundos']#OK

['Cruzamento 1: 5 minutos e 27 segundos', 'Cruzamento 2: 9 minutos e 25 segundos', 'Cruzamento 3: 10 minutos e 12 segundos', 'Cruzamento 4: 12 minutos e 54 segundos', 'Cruzamento 5: 12 minutos e 59 segundos', 'Cruzamento 6: 13 minutos e 34 segundos', 'Cruzamento 7: 15 minutos e 2 segundos', 'Cruzamento 8: 18 minutos e 24 segundos', 'Cruzamento 9: 19 minutos e 49 segundos', 'Cruzamento 10: 26 minutos e 33 segundos', 'Cruzamento 11: 27 minutos e 5 segundos', 'Cruzamento 12: 32 minutos e 13 segundos', 'Cruzamento 13: 32 minutos e 52 segundos', 'Cruzamento 14: 32 minutos e 58 segundos', 'Cruzamento 15: 33 minutos e 43 segundos', 'Cruzamento 16: 34 minutos e 43 segundos', 'Cruzamento 17: 34 minutos e 57 segundos', 'Cruzamento 18: 41 minutos e 29 segundos', 'Cruzamento 19: 44 minutos e 41 segundos']


: 